# Merging (averaging) two submissions and optimize the leaderboard metric F-score

In [3]:
# Importing a soulution shared on the forum from a top competitor. Thanks for sharing! 
# We will going to merge this one with mine.
import pandas as pd

data_0 = pd.read_pickle('prediction_from_other_solution/prediction_arboretum.pkl')

In [5]:
data_0.shape

(4833292, 3)

In [6]:
data_0.order_id.nunique()

75000

In [14]:
# This is the fansatic piece of code to optimized our leaderboard metric --> F score. 
# This is from one of the best data scientiest in the world. Faron.Thanks for sharing 

class F1Optimizer():
    def __init__(self):
        pass

    @staticmethod
    def get_expectations(P, pNone=None):
        expectations = []
        P = np.sort(P)[::-1]

        n = np.array(P).shape[0]
        DP_C = np.zeros((n + 2, n + 1))
        if pNone is None:
            pNone = (1.0 - P).prod()

        DP_C[0][0] = 1.0
        for j in range(1, n):
            DP_C[0][j] = (1.0 - P[j - 1]) * DP_C[0, j - 1]

        for i in range(1, n + 1):
            DP_C[i, i] = DP_C[i - 1, i - 1] * P[i - 1]
            for j in range(i + 1, n + 1):
                DP_C[i, j] = P[j - 1] * DP_C[i - 1, j - 1] + (1.0 - P[j - 1]) * DP_C[i, j - 1]

        DP_S = np.zeros((2 * n + 1,))
        DP_SNone = np.zeros((2 * n + 1,))
        for i in range(1, 2 * n + 1):
            DP_S[i] = 1. / (1. * i)
            DP_SNone[i] = 1. / (1. * i + 1)
        for k in range(n + 1)[::-1]:
            f1 = 0
            f1None = 0
            for k1 in range(n + 1):
                f1 += 2 * k1 * DP_C[k1][k] * DP_S[k + k1]
                f1None += 2 * k1 * DP_C[k1][k] * DP_SNone[k + k1]
            for i in range(1, 2 * k - 1):
                DP_S[i] = (1 - P[k - 1]) * DP_S[i] + P[k - 1] * DP_S[i + 1]
                DP_SNone[i] = (1 - P[k - 1]) * DP_SNone[i] + P[k - 1] * DP_SNone[i + 1]
            expectations.append([f1None + 2 * pNone / (2 + k), f1])

        return np.array(expectations[::-1]).T

    @staticmethod
    def maximize_expectation(P, pNone=None):
        expectations = F1Optimizer.get_expectations(P, pNone)

        ix_max = np.unravel_index(expectations.argmax(), expectations.shape)
        max_f1 = expectations[ix_max]

        predNone = True if ix_max[0] == 0 else False
        best_k = ix_max[1]

        return best_k, predNone, max_f1

    @staticmethod
    def _F1(tp, fp, fn):
        return 2 * tp / (2 * tp + fp + fn)

    @staticmethod
    def _Fbeta(tp, fp, fn, beta=1.0):
        beta_squared = beta ** 2
        return (1.0 + beta_squared) * tp / ((1.0 + beta_squared) * tp + fp + beta_squared * fn)


In [15]:

def create_products_faron(df):
    products = df.product_id.values
    prob = df.prediction.values

    sort_index = np.argsort(prob)[::-1]
    prob = prob[sort_index]
    products = products[sort_index]

    opt = F1Optimizer.maximize_expectation(prob)
    
    best_prediction = ['None'] if opt[1] else []
    best_prediction += [str(p) for p in products[:opt[0]]]
    f1_max = opt[2]

    best = ' '.join(best_prediction)
    return (df.iloc[0,0], best)

In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools
from tqdm import tqdm

from subprocess import check_output
print(check_output(["ls","input"]).decode("utf8"))
# Setting working directory


aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



In [10]:
# Let's try to use the Faron F optimizer on the arboretum sharing solution to see 
# how much this score on leaderboard. 
if __name__ == '__main__':
    data = pd.read_pickle('prediction_from_other_solution/prediction_arboretum.pkl')

    data = data.loc[data.prediction > 0.011, ['order_id', 'prediction', 'product_id']]
    print (data)
    
    out = [create_products_faron(group) for name, group in tqdm(data.groupby(data.order_id))]

    data = pd.DataFrame(data=out, columns=['order_id', 'products'])

    data.to_csv('sub.csv', index=False)
# This was pretty good!

         order_id  prediction  product_id
0           65432    0.027229       17330
1          203668    0.048930       17330
2         1734476    0.044416       17330
3         1627971    0.048180       17330
4         1213682    0.027965       17330
8          761818    0.029585       17330
9         1029203    0.018535       17330
10        3052061    0.042104       17330
11        1981837    0.060045       17330
12        2224210    0.357404       17330
13        2669437    0.053260       17330
14         526558    0.211673       17330
15        1339380    0.160464       17330
17         993166    0.357754       17330
19        1320217    0.064478       17330
20        1697635    0.081522       17330
22        2794524    0.613382       17330
23        2528159    0.015822       17330
24         290583    0.019544       17330
25         552080    0.638703       17330
26        2110203    0.138697       17330
27         347183    0.021586       17330
28         581992    0.123443     

100%|██████████| 75000/75000 [1:57:18<00:00,  5.50it/s]  


In [11]:
data.shape 

(75000, 2)

In [ ]:
# Let's retrieve our solution and pass it into the Faron F optimazer.
if __name__ == '__main__':
    data = pd.read_pickle('/prediction_lgbm.pkl')

    data = data.loc[data.prediction > 0.0099, ['order_id', 'prediction', 'product_id']]
    print (data)
    
    out = [create_products_faron(group) for name, group in tqdm(data.groupby(data.order_id))]

    data = pd.DataFrame(data=out, columns=['order_id', 'products'])

    data.to_csv('sub.csv', index=False)
    data.shape

In [10]:
#merging two submissions.
import pandas as pd
data_1 = pd.read_pickle('prediction_from_other_solution/prediction_arboretum.pkl')
data_2 = pd.read_pickle('prediction_lgbm.pkl')

data_1.reset_index(inplace=True)
data_2.reset_index(inplace=True)

print (data_1.shape)
print (data_2.shape)

(4833292, 4)
(4833292, 4)


In [11]:
# Here averaging the two solutions
data_merged = pd.merge (data_1 , data_2, on = ['order_id' , 'product_id'], how = 'outer') # outer should give same lenght if data are big the same
print ( data_merged.shape)

data_merged ['prediction'] = (data_merged ['prediction_x'] + data_merged ['prediction_y']) / 2
data_merged.head()

(4833292, 6)


,index_x,order_id,prediction_x,product_id,index_y,prediction_y,prediction
0,0,65432,0.027229,17330,0,0.035937,0.031583
1,1,203668,0.048930,17330,1,0.033004,0.040967
2,2,1734476,0.044416,17330,2,0.033722,0.039069
3,3,1627971,0.048180,17330,3,0.037862,0.043021
4,4,1213682,0.027965,17330,4,0.031600,0.029782


In [12]:
#To delete the column without having to reassign df you can do:
data_merged.drop( ['prediction_x','prediction_y','index_x','index_y',] , axis=1, inplace=True) 
print (data_merged.shape)
data_merged.head()


(4833292, 3)


,order_id,product_id,prediction
0,65432,17330,0.031583
1,203668,17330,0.040967
2,1734476,17330,0.039069
3,1627971,17330,0.043021
4,1213682,17330,0.029782


In [12]:
## This is the final submission that put me in the top 12%, just 20ish position short from the bronze medal
## There were thousand of participants for this one.


if __name__ == '__main__':
   
    data_merged = data_merged.loc[data_merged.prediction > 0.011, ['order_id', 'prediction', 'product_id']]
    print (data_merged)
    
    out = [create_products_faron(group) for name, group in tqdm(data_merged.groupby(data_merged.order_id))]

    data_merged = pd.DataFrame(data_merged=out, columns=['order_id', 'products'])

    data_merged.to_csv('merged_sub.csv', index=False)
    data_merged.shape

         order_id  prediction  product_id
0           65432    0.027229       17330
1          203668    0.048930       17330
2         1734476    0.044416       17330
3         1627971    0.048180       17330
4         1213682    0.027965       17330
8          761818    0.029585       17330
9         1029203    0.018535       17330
10        3052061    0.042104       17330
11        1981837    0.060045       17330
12        2224210    0.357404       17330
13        2669437    0.053260       17330
14         526558    0.211673       17330
15        1339380    0.160464       17330
17         993166    0.357754       17330
19        1320217    0.064478       17330
20        1697635    0.081522       17330
22        2794524    0.613382       17330
23        2528159    0.015822       17330
24         290583    0.019544       17330
25         552080    0.638703       17330
26        2110203    0.138697       17330
27         347183    0.021586       17330
28         581992    0.123443     

100%|██████████| 75000/75000 [1:59:41<00:00,  5.03it/s]  
